# Stress molecules

**This program was created as a part of my master's thesis that deals with a relationship between interspecies gene variants of cellular stress response and longevity.**

**When downloaded, the Jupyter notebook includes also outputs of the run that was proceeded for the master's thesis analysis. The only difference is that the BRCA1 and CAMK2G genes were excluded from the final dataset, as an error was encountered in obtaining information about them in each run of the program. This error sometimes occurred for other genes as well, and occurred mainly in the evening. The error was not resolved, but my explanation is that there was server overload. The dataset for the thesis was obtained in parts by part 7 of this program.**


### It consists of:
<ol>
    <li> <strong>Get a list of stress molecule symbols</strong>
        <ul>
            <li> downloading list of signaling molecules of pathways from Reactome database selected by user</li>
        </ul>
    </li>

<li><strong>Get orthologs for every stress molecule; from NCBI</strong>
    <ul>
        <li> sequence acquisition of orthologs of these molecules, i.e. genes, from NCBI databases (= fasta files)
        <li> parsing the fasta files and creating of various tables with information about these genes an its sequences
        <li> checking of for how many organisms the data has been downloaded and selecting the data that upcoming analysis will be performed with
    </ul>
</li>
    
    
<li><strong>Animal Traits & AnAge databases; calculating longevity quotient</strong>
    <ul>
        <li> merging available datasets of characteristics of organisms
        <li> calculationg longevity quotient
        <li> selecting short- and long-living organisms
    </ul>
</li>
    
<li><strong>Select reference sequences</strong>
    <ul>
        <li> selecting reference sequences, i.e. creating the final dataset
    </ul>
</li>
    
<li><strong>Create files for CAASTools Discovery analysis</strong>
    <ul>
        <li> creating fasta files for multiple sequence alignment (MSA) and a configuration file with short- and long-living organisms
        <li> performing MSA
    </ul>
</li> 
    
<li><strong>Create csv files</strong>
    <ul>
        <li> creating csv files with results 
    </ul>
</li>
    

<li> Additional: Reference sequences selection if NCBI connection error
    <ul>
        <li> sometimes an error occur, and unfortunatelly it was not resolved, for that reaseon, there is slower running script, that may help)
    </ul>
</li>
</ol>

### Import path

In [1]:
win_path = str(input('Type in the path of the parental folder (for "fpilarova_masters-main" it would be C:\\\\Users\\\\frant\\\\Desktop\\\\fpilarova_masters-main):\n'))

main_path = win_path.replace('\\\\', '\\')

Type in the path of the parental folder (for "fpilarova_masters-main" it would be C:\\Users\\frant\\Desktop\\fpilarova_masters-main):
 C:\\Users\\frant\\Desktop\\fpilarova_masters-main


### Import modules

In [45]:
import httplib2 as http
import json
import os
import csv
import re
import math
import pandas as pd
import fastaparser
from collections import Counter
from urllib.request import urlopen
import requests
from itertools import islice
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry


try:
    from urlparse import urlparse
except ImportError:
    from urllib.parse import urlparse

import warnings
warnings.filterwarnings(action="ignore", message="unclosed", category=ResourceWarning)

### Create subfolders

In [ ]:
os.system('mkdir {0}\\results'.format(win_path))
os.system('mkdir {0}\\CAAStools_files'.format(win_path))
os.system('mkdir {0}\\NCBI_Datasets_download'.format(win_path))

## 1 Get a list of stress molecule symbols
**Description of the script:**
1. List of gene symbols is downloaded; for every pathway requested (input: Reactome ID)
2. The symbols are checked to be HGNC-approved
    - if not -> printed (EDIT - ask HGNC for HGNC-approved name of that gene)
3. Checked symbols are added to the final list (stress_molecules_list = containing stress molecules of all requested Reactome IDs)
5. Table gene_in_multiple_paths_check is created; with information about in which Reactome pathways the genes are involved

### Variables

In [3]:
stress_molecules_list = []
#stress_molecules_list = ['AAAS', 'HSF1'] #TEST
# final list of all stress molecules; to be saved as stress_molecules.csv
      
reactome_ids_symbols_dictionary = {}
# dictionary of Reactome ids and their HGNC-approved gene symbols
# {Reactome_ID_1: [gene_1, gene_2], ...}

gene_in_multiple_paths_check = {}
# dictionary of genes and Reactome IDs in which the genes are included
# {gene_1: [Reactome_ID_1, Reactome_ID_2], ...} 

HGNC_not = []
# list of gene symbols that are not HGNC-approved

### Script: Ask user for Reactome path IDs
**Input:**
- reactome_ids = list of Reactome IDs
- reactome_names (optional)


**Testing input:**

- R-HSA-3371556 R-HSA-9612973 R-HSA-9711097 R-HSA-381119 R-HSA-73894
- hsr autophagy starvation upr dna-repair




In [5]:
# ask for input

reactome_ids_input = str(input(' Enter Reactome path IDs (with spaces as separators):')) # ask for IDs
reactome_ids = reactome_ids_input.split(' ')
pathway_names_input = str(input('\n Do you want to name them? \n Yes - enter the names (with spaces as separators) \n No - press enter \n')) #ask for names if needed
pathway_names = pathway_names_input.split(' ')

while (len(reactome_ids) != len(pathway_names)) and (pathway_names[0] != ''): # check if there is the same number of names and ids
    print(' Error: there is different number of path IDs and path names.')
    reactome_ids_input = str(input(' Enter Reactome path IDs (with spaces as separators): '))
    reactome_ids = reactome_ids_input.split(' ')
    pathway_names_input = str(input('Do you want to name them? \n Yes - enter the names (with spaces as separators) \n No - press enter \n'))
    pathway_names = pathway_names_input.split(' ')


# inform about requested IDs

print('\n List of Reactome IDs saved to be examined later:') 
for i in range(len(reactome_ids)):                                           
    if pathway_names[0] != '':
        print(reactome_ids[i], ' (', pathway_names[i], ')', sep='')
    else:
        print(reactome_ids[i])

print('\n')

 Enter Reactome path IDs (with spaces as separators): R-HSA-3371556 R-HSA-9612973 R-HSA-9711097 R-HSA-381119 R-HSA-73894

 Do you want to name them? 
 Yes - enter the names (with spaces as separators) 
 No - press enter 
 hsr autophagy starvation upr dna-repair



 List of Reactome IDs saved to be examined later:
R-HSA-3371556 (hsr)
R-HSA-9612973 (autophagy)
R-HSA-9711097 (starvation)
R-HSA-381119 (upr)
R-HSA-73894 (dna-repair)




### Script: Download the symbols from Reactome and check if they are HGNC-approved
**Description of the script:**
1. Gene symbols from Reactome are downloaded (using Reactome IDs requested earlier)
2. The symbols are checked to be HGNC-approved
3. List stress_molecules_list is created; containing HGNC-approved symbols, each only once
4. Dictionary 'gene_in_multiple_paths_check' is generated to identify genes that are involved in multiple Reactome pathways.

Results:
1. stress_molecules_list
    - stress_molecules.csv
2. gene_in_multiple_paths_check 
    - gene_in_multiple_paths_check.csv

In [6]:
for k in range(len(reactome_ids)): # for every Reactome ID requested:
    
    if pathway_names[0] != '':    
        print('Gene symbols of pathway: {0} (ID: {1}) are being downloaded and checked...'.format(pathway_names[k], reactome_ids[k]))
    else: 
        print('Gene symbols of pathway with ID: {} are being downloaded and checked...'.format(reactome_ids[k]))
    
    
    reactome_id = reactome_ids[k]
    
    working_symbols_list = []        # list of gene symbols of current Reactome ID to be HGNC-checked
    
    HGNC_approved_symbols_list = []
    
    
    
    # download a list of molecules using Reactome API 
    
    headers = {'Accept': 'application/json'}

    uri = 'https://reactome.org/ContentService/data/participants'
    path = '/{}'.format(reactome_id)

    target = urlparse(uri+path)
    method = 'GET'
    body = ''

    h = http.Http()

    response, content = h.request(
        target.geturl(),
        method,
        body,
        headers)


    if response['status'] == '200':    # assume that content is a json reply
        data = json.loads(content)     # parse content with the json module 


        for i in range(len(data)):
            for j in range(len(data[i]['refEntities'])):
                if 'UniProt' in data[i]['refEntities'][j]['displayName']:         # choose only UniProt outcomes; i.e. gene symbols
                    symbol = data[i]['refEntities'][j]['displayName'].split()[1]
                    if symbol not in working_symbols_list:                        # check that every symbol is only once
                        working_symbols_list.append(symbol)

    else:
        print( 'Error detected: ' + response['status'])
           
    
    
    # with using HGNC API find out if symbols are HGNC-approved

    if len(working_symbols_list) != 0:                 # if list is not empty
        for symbol in working_symbols_list:            # check every symbol in list
            
            # set the url
            headers = {'Accept': 'application/json'}

            uri = 'https://rest.genenames.org'
            path = '/search/symbol/{}'.format(symbol)

            target = urlparse(uri+path)
            method = 'GET'
            body = ''

            h = http.Http()

            response, content = h.request(
                target.geturl(),
                method,
                body,
                headers)

            if response['status'] == '200':            # assume that content is a json reply
                data = json.loads(content)             # parse content with the json module 
                check = data['response']['numFound']   # check can be 0 (not HGNC-approved) or 1 (approved)

                if (check == 0) and (symbol not in HGNC_not):
                    HGNC_not.append(symbol)
                elif (check == 1) and (symbol not in HGNC_approved_symbols_list):
                    HGNC_approved_symbols_list.append(symbol)

            else:
                print( 'Error detected: ' + response['status'])
        
        reactome_ids_symbols_dictionary[reactome_id] = HGNC_approved_symbols_list      # 'reactome_id_1: [gene_1, gene2, ...]' is added to dictionary
        

    else: 
        print('Error: there is no data for {}.'.format(reactome_id))
    
    for gene in HGNC_approved_symbols_list:
        if gene not in stress_molecules_list: 
            stress_molecules_list.append(gene)  # HGNC-approved symbols added to list of stress molecules (--> stress_molecules.csv)
        
        if gene not in gene_in_multiple_paths_check.keys():
            gene_in_multiple_paths_check.update({gene: [reactome_id]})
        else:
            gene_in_multiple_paths_check[gene].append(reactome_id)
        


# print symbols that are not HGNC-approved

if len(HGNC_not) != 0:    
    print('\n Symbols not HGNC-approved:', sep = ' ')
    for gene in HGNC_not:
        print(gene, sep = ' ')

        

df_stress_molecules = pd.DataFrame(stress_molecules_list, columns = ['symbol'])
    
multiple_paths_df = pd.DataFrame.from_dict(gene_in_multiple_paths_check, orient = 'index')


print('\n Done.')

Gene symbols of pathway: hsr (ID: R-HSA-3371556) are being downloaded and checked...
Gene symbols of pathway: autophagy (ID: R-HSA-9612973) are being downloaded and checked...
Gene symbols of pathway: starvation (ID: R-HSA-9711097) are being downloaded and checked...
Gene symbols of pathway: upr (ID: R-HSA-381119) are being downloaded and checked...
Gene symbols of pathway: dna-repair (ID: R-HSA-73894) are being downloaded and checked...

 Done.


### Results: List of stress molecules

In [7]:
df_stress_molecules

,symbol
0,EP300
1,CREBBP
2,HSPA1L
3,HSPA2
4,HSPA8
...,...
746,MSH2
747,MSH6
748,MLH1
749,PMS2


### Results: Multiple paths check
To identify genes that are involved in multiple Reactome pathways.

In [8]:
multiple_paths_df

,0,1,2
EP300,R-HSA-3371556,R-HSA-73894,None
CREBBP,R-HSA-3371556,None,None
HSPA1L,R-HSA-3371556,None,None
HSPA2,R-HSA-3371556,None,None
HSPA8,R-HSA-3371556,R-HSA-9612973,None
...,...,...,...
MSH2,R-HSA-73894,None,None
MSH6,R-HSA-73894,None,None
MLH1,R-HSA-73894,None,None
PMS2,R-HSA-73894,None,None


## 2 Get orthologs for every "stress molecule"; from NCBI
**Description of the script:** 
1. Download and save FASTA files using NCBI Datasets
    - FASTA file (orthologs) for every stress molecule using NCBI Datasets (datasets.exe) is downloaded and saved
2. Parse FASTA files
    - FASTA files are parsed and saved to: 
        - orthologs_list with all information from FASTA file saved
        - orthologs_count for analysis of count of sequences per gene id
        - genes_organisms_dictionary for genes-organisms check
        - all_orthologs_refseqs_sorted for selection of one sequence per gene id
3. Genes-organisms check



### Download and save FASTA files using NCBI Datasets

#### Script

In [9]:
print('FASTA files for every stress molecule containing data with orthologous sequences are being downloaded and saved... \n') 

for stress_molecule in stress_molecules_list:
    os.system('datasets download ortholog symbol {1} --taxon-filter mammals --exclude-rna --exclude-gene --filename {0}\\NCBI_Datasets_download\\{1}.zip'.format(win_path, stress_molecule))
    os.system('cd {0}\\NCBI_Datasets_download\\ & tar -xf {0}\\NCBI_Datasets_download\\{1}.zip'.format(win_path, stress_molecule))
    os.system('move {0}\\NCBI_Datasets_download\\ncbi_dataset\\data\\protein.faa {0}\\NCBI_Datasets_download'.format(win_path))
    os.system('ren {0}\\NCBI_Datasets_download\\protein.faa {1}.faa'.format(win_path, stress_molecule))
os.system('mkdir {0}\\NCBI_Datasets_download\\hej'.format(win_path))
os.system('move {0}\\NCBI_Datasets_download\\README.md {0}\\NCBI_Datasets_download\\hej'.format(win_path))
os.system('move {0}\\NCBI_Datasets_download\\*.zip {0}\\NCBI_Datasets_download\\hej'.format(win_path))
os.system('rmdir /s /q {0}\\NCBI_Datasets_download\\hej'.format(win_path))
os.system('rmdir /s /q {0}\\NCBI_Datasets_download\\ncbi_dataset'.format(win_path))


files = os.listdir(r'{0}\NCBI_Datasets_download'.format(main_path))
if '.ipynb_checkpoints' in files: files.remove('.ipynb_checkpoints')


for stress_molecule in stress_molecules_list:
    if str(stress_molecule) + '.faa' not in files:
        print('Error: a file for', stress_molecule, 'was not downloaded. \n')
        stress_molecules_list.remove(stress_molecule)

print('Done.')


FASTA files for every stress molecule containing data with orthologous sequences are being downloaded and saved... 

Error: a file for HSPA7 was not downloaded. 

Error: a file for GABARAPL3 was not downloaded. 

Error: a file for H2BC12L was not downloaded. 

Error: a file for H2BC13 was not downloaded. 

Done.


### Parse FASTA files

#### Variables and functions

In [10]:
# GENERAL VARIABLES
gene_ids = [] 
# gene ids of all orthologs (of all stress molecules together)

orthologs_list = [] 
# list of all orthologous sequences and information about them
# [{human_gene_name: X, gene id: X, gene name: X, organism name; X, accession_number (that is refseq number): X, protein sequence: X}, {...}]
# -> orthologs.csv


# TO SEE HOW MANY REFERENCE SEQUENCES THERE ARE FOR ONE GENE ID
orthologs_count_list = []
# list of genes and count of its sequences (refseq) downloaded from NCBI 
# [{human_gene_name: x, gene_id: x, seq_count = x, most_repres_seq_count = x, unique_seq_count = x}, {...}, ...]

already_been_there = []
# working list


# FOR GENES-ORGANISMS CHECK
genes_organisms_dictionary = {}



# FOR SELECTION OF ONE SEQUENCE PER GENE ID
all_orthologs_refseqs_sorted = {}
# {gene_id1: 
#     {RefSeq1: 
#          {protein_sequence: X, human_gene_name: X, gene_name: X, organism: X}, 
#      RefSeq2: {...}
#     }, 
#  gene_id2: 
#      {RefSeq3: {}
#     }, 
#  ...}

In [11]:
def most_frequent_count(List):
    occurence_count = Counter(List)
    return occurence_count.most_common(1)[0][1]

def most_frequent_name(List):
    occurence_count = Counter(List)
    return occurence_count.most_common(1)[0][0]

#### Script: Parse FASTA files and save the information into lists and dictionaries

In [12]:
# parse all FASTA files downloaded 
# every file is named as the gene which orthologs seqs it contains of

files = os.listdir(r'{0}\NCBI_Datasets_download'.format(main_path))
if '.ipynb_checkpoints'in files: files.remove('.ipynb_checkpoints')

for file in files:
    
    gene_name = file.replace('.faa', '')

    orthologs_count_dictionary = {}   # working dictionary; will be used for "seqs count check" 
                                      # {gene_id1: [seq1, seq2, seq3], gene_id2: [seq1, seq2], ...}
    
    organism_gene_id_dict = {}        # working dictionary; will be used for genes-organisms check
        
    with open(r'{0}\NCBI_Datasets_download\{1}'.format(main_path, file)) as fasta_file:
        parser = fastaparser.Reader(fasta_file)
        
        for seq in parser: # for every record (ortholog) of FASTA file        
            
            # split the FASTA record 
            working_list = seq.description.split(" [")     
            gene_id = (re.findall('\\d+', working_list[2])[0])
            gene_ids.append(gene_id)


            
            ### for orthologs.csv ###
            # add seq (ortholog) atributes to orthologs_list
            
            ortholog_dictionary = {}   # {human_gene_name: X, gene id: X, gene name: X, organism name; X, accession number: X, protein sequence: X}
            
            ortholog_dictionary['human_gene_name'] = gene_name                              # human gene name added
            ortholog_dictionary['gene_name'] = working_list[0]                              # gene name added
            organism_name = working_list[1][9:].replace("]", "")
            organism_name = organism_name.replace(" ", "_")
            ortholog_dictionary['organism'] = organism_name                                 # organism name added
            ortholog_dictionary['gene_id'] = gene_id                                        # gene ID added
            ortholog_dictionary['accession_number'] = seq.id                                # accession number added 
            ortholog_dictionary['protein_sequence'] = seq.sequence_as_string()              # protein sequence with its accession number added

            orthologs_list.append(ortholog_dictionary)
            # orthologs_list = [{human_gene_name: X, gene id: X, gene name: X, organism name; X, accession number: X, protein sequence: X}, {...}]
            
            

            ### for orthologs_count.csv and final_orthologs_seqs.csv ###     
            # add sequence to orthologs_count_dictionary {gene_id1: [seq1, seq2, seq3], gene_id2: [seq1, seq2], ...}
            # add RefSeq number and sequence to all_orthologs_refseqs_sorted dictionary {gene_id1: {RefSeq1: seq1, RefSeq2: seq2, ...}, gene_id2: {}, ...}
            
            if gene_id not in already_been_there: 
                first_sequence = seq.sequence_as_string()
                orthologs_count_dictionary[gene_id] = [first_sequence]
                all_orthologs_refseqs_sorted[gene_id] = {}
                all_orthologs_refseqs_sorted[gene_id].update({seq.id: {'protein_sequence': seq.sequence_as_string(), 'human_gene_name': gene_name, 'gene_name': working_list[0], 'organism': working_list[1][9:].replace("]", "")}})
                
            else:
                another_sequence = seq.sequence_as_string()
                orthologs_count_dictionary[gene_id].append(another_sequence)
                all_orthologs_refseqs_sorted[gene_id].update({seq.id: {'protein_sequence': seq.sequence_as_string(), 'human_gene_name': gene_name, 'gene_name': working_list[0], 'organism': working_list[1][9:].replace("]", "")}})

            already_been_there.append(gene_id)
            
            
            
            ### for genes_organisms_check.csv ###
            organism_gene_id_dict[organism_name] = gene_id      

            
        genes_organisms_dictionary.update({gene_name : organism_gene_id_dict})
        # {gene_name1: {organism1: gene_id1, organism2: gene_id2, ...}, gene_name2: {}, ...}
            
            
    
    ### for orthologs_count.csv ###     
    # create dictionary with gene ids and their protein sequence counts (and the most represented sequence)
    
    for key in orthologs_count_dictionary.keys():
        ortholog = {}
        ortholog['human_gene_name'] = file.strip('.faa')
        ortholog['gene_id'] = str(key)
        ortholog['seq_count'] = len(orthologs_count_dictionary[key])
        ortholog['most_repres_seq_count'] = most_frequent_count(orthologs_count_dictionary[key])
        
        counter_keys = Counter(orthologs_count_dictionary[key])
        ortholog['unique_seq_count'] = len(counter_keys.keys())
        
        orthologs_count_list.append(ortholog)
        # orthologs_count_list = [{human_gene_name: 'xxx.faa', gene_id: x, seq_count = x, most_repres_seq_count = x, unique_seq_count = x}, {...}, ...]


df_orthologs_list = pd.DataFrame.from_dict(orthologs_list)
df_ortholog_count_list = pd.DataFrame.from_dict(orthologs_count_list)



#### Results: List of all orthologs downloaded 

In [13]:
df_orthologs_list

,human_gene_name,gene_name,organism,gene_id,accession_number,protein_sequence
0,AAAS,AAAS,Equus_caballus,100063811,XP_001494960.1,MCSLGLFPPPPPRGQVTLYEHNNELVTGNSYESPPPDFRGQWINLP...
1,AAAS,AAAS,Equus_caballus,100063811,XP_023499419.1,MRVAPSFSPVHSTGRDVGLFGVLNEIANSEEEVFEWVKTASSWALA...
2,AAAS,AAAS,Sus_scrofa,100154333,XP_003355451.1,MCSLGLFPPPPPRGQVTLYEHNNELVTGSSYESPPPDFRGQWINLP...
3,AAAS,AAAS,Oryctolagus_cuniculus,100356621,XP_002711061.1,MCSLGLFPPPPPMGQVTLYEHNNELVTGSSYESPPPDFRGQWINLP...
4,AAAS,AAAS,Oryctolagus_cuniculus,100356621,XP_008254733.1,MARGLPSSITGSKCGKDASTRVFEWVKTASSWALALCRWASSLHGS...
...,...,...,...,...,...,...
359500,ZNF830,Zfp830,Rattus_norvegicus,497967,NP_001032437.1,MASSTSARTPAGKRVVNQEELRRLMKEKQRLSTNRKRIESPFAKYN...
359501,ZNF830,ZNF830,Bos_taurus,539497,NP_001076081.1,MASSASARPPAGKRVVNQDELRRLMKEKQRLSTNRKRIESPFAKYN...
359502,ZNF830,Zfp830,Mus_musculus,66983,NP_080160.2,MASSTSTRTPAGKRVVNQEELRRLMREKQRLSTNRKRIESPFAKYN...
359503,ZNF830,ZNF830,Macaca_mulatta,715130,XP_001113549.3,MPRRTETRIALGLVAKMASSASARTPAGKRVVNQEELRRLMKEKQR...


#### Results: Count of reference sequences of every orthologous gene


In [14]:
df_ortholog_count_list

,human_gene_name,gene_id,seq_count,most_repres_seq_count,unique_seq_count
0,AAAS,100063811,2,1,2
1,AAAS,100154333,1,1,1
2,AAAS,100356621,4,3,2
3,AAAS,100405154,2,1,2
4,AAAS,100444879,4,1,4
...,...,...,...,...,...
138911,ZNF830,497967,1,1,1
138912,ZNF830,539497,1,1,1
138913,ZNF830,66983,1,1,1
138914,ZNF830,715130,1,1,1


### Genes-organisms check


#### Script

In [15]:
# create DataFrame from genes_organisms_dictionary

df_genes_organisms = pd.DataFrame(list(genes_organisms_dictionary.values()), index=list(genes_organisms_dictionary.keys()))

# variables

genes_count = len(genes_organisms_dictionary.keys())
organisms_count = 0
count_organisms_seq_ratio = {}
count_genes_seq_ratio = {}

# for which organisms there is enough data? 
# i.e. ratio of nonexisting vs existing ortholog seqs for every organism
for column in df_genes_organisms:
    organisms_count += 1
    count_organisms_seq_ratio.update({column: df_genes_organisms[column].isnull().sum()/genes_count})

# for which genes there is enough data? 
# i.e. ratio of nonexisting vs existing seqs for every gene
for key in genes_organisms_dictionary.keys():
    nan_count = df_genes_organisms.loc[[key]].isna().sum().sum()
    count_genes_seq_ratio.update({key: nan_count/organisms_count})



# pop an organism or a gene from the DataFrame if there is not enough data (seqs) for them 

organisms_not_enough = []
genes_not_enough = []
gene_ids_to_pop = [] # list of gene ids to pop from all_orthologs_refseq_sorted (aka list for future analysis)

for key in count_organisms_seq_ratio.keys():
    if count_organisms_seq_ratio[key] > 0.2:
        organisms_not_enough.append(key)
        

# Remove the organism with too litle data from the analysis
for organism in organisms_not_enough:
    gene_ids_to_pop += df_genes_organisms[organism].to_list()
    df_genes_organisms.pop(organism)
    
    

for key in count_genes_seq_ratio.keys():
    if count_genes_seq_ratio[key] > 0.2:
        genes_not_enough.append(key)
        
        
# Remove the gene with too litle data from the analysis
for gene in genes_not_enough:
    gene_ids_to_pop += df_genes_organisms.loc[gene].tolist()
    df_genes_organisms.drop(gene, inplace=True)



# print genes and organisms which enough data do not exist for

if len(genes_not_enough) != 0:
    print('There is not enough data for genes:')
    for gene in genes_not_enough:
        print(gene)

if len(organisms_not_enough) != 0:
    print('There is not enough data for organisms:')
    for organism in organisms_not_enough:
        print(organism)
        
print('\n They are not included in future analysis.')



# remove organisms and genes with not enough data from all_orthologs_refseqs_sorted
gene_ids_to_pop = [b for b in gene_ids_to_pop if isinstance(b, str)] # remove all nan


for geneid in gene_ids_to_pop:
    del all_orthologs_refseqs_sorted[geneid]



There is not enough data for genes:
CCL2
CENPS
CHMP3
ERCC5
GTF2H2
H2AB1
H2AC14
H2AC18
H2AC20
H2AC4
H2AC6
H2AC7
H2BC1
H2BC11
H2BC12
H2BC14
H2BC15
H2BC17
H2BC21
H2BC26
H2BC3
H2BC4
H2BC5
H2BC9
H3-4
H4C1
HBB
HSPA1A
HSPA1B
HSPA1L
HSPA6
ISY1
MAP1LC3C
PLA2G4B
POM121
POM121C
RANBP2
RHNO1
RNASE1
RPL10L
RPL17
RPL36A
RPL39L
RPL41
RPS10
RPS4Y1
RPS4Y2
RRAGB
SEM1
SRPRB
SULT1A3
TUBA1A
TUBA1B
TUBA1C
TUBA3C
TUBA3D
TUBA3E
TUBA4A
TUBA4B
TUBB2A
TUBB2B
TUBB3
TUBB8
TUBB8B
UBB
UBC
There is not enough data for organisms:
Elephantulus_edwardii

 They are not included in future analysis.


#### Results: Final table of genes, organism and corresponding gene ids

In [16]:
df_genes_organisms

,Equus_caballus,Sus_scrofa,Oryctolagus_cuniculus,Callithrix_jacchus,Pongo_abelii,Ailuropoda_melanoleuca,Nomascus_leucogenys,Loxodonta_africana,Cavia_porcellus,Cricetulus_griseus,...,Mus_musculus,Rattus_norvegicus,Pan_troglodytes,Bos_taurus,Canis_lupus_familiaris,Macaca_mulatta,Homo_sapiens,Monodelphis_domestica,Sturnira_hondurensis,Microtus_oregoni
AAAS,100063811,100154333,100356621,100405154,100444879,100468100,100586295,100662257,100727054,100767976,...,223921,300259,467003,506561,607867,719095,8086,NaN,NaN,NaN
ABL1,100069715,100524544,100345909,100398414,100453216,100466468,100601002,100677152,100720976,100768098,...,11350,311860,107976801,540876,491292,722449,25,100619549,118980343,121462621
ABRAXAS1,100061483,100524450,100350582,100401953,100449014,100463877,100597649,100675954,100721329,100766855,...,70681,289468,461205,504796,478459,713263,84142,100017569,118979544,121440920
ACADVL,100061583,100520636,100346654,100399197,100454381,100472785,100585921,100657276,100713909,100767527,...,11370,25363,455237,282130,489463,714326,37,100016619,118974685,121456541
ACD,100066184,100623181,100346875,100405339,100447506,100474125,100586139,100661748,100730199,100763518,...,497652,307798,454170,510353,487931,699230,65057,103105155,118983853,121447226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YIF1A,100052354,100521064,127489740,100394171,100443779,100482492,100597699,100656055,100727936,100763610,...,68090,171441,746258,506336,476014,714486,10897,100029722,118991769,121454377
YWHAE,100059962,100620600,NaN,100400280,100455220,100484851,100600918,100658987,100714396,NaN,...,22627,29753,454409,282125,480645,720156,7531,100012276,119003777,121456736
YY1,100630389,102162035,103346032,100411814,100447742,100474192,100583435,100673864,100730873,100767279,...,22632,24919,453159,534353,611171,717370,7528,100022685,118976930,121434266
ZBTB17,100050436,100620358,100348036,100398482,100454123,100482510,100589356,100654604,100731568,100754046,...,22642,313666,740453,512915,607083,696327,7709,100619919,118986310,121448060


## 3 Animal Traits & AnAge databases; calculating longevity quotient

**Description of script:** Edits data from Animal Traits dataset and saves name of an organism and its body mass to table 'df_animaltraits'. Merges AnAge data and Animal Traits data (table 'df_anage_animaltraits'); and calculate longevity quotient. 

#### Script: Load Animal Traits dataset

In [17]:
df_animaltraits = pd.read_csv(r'{0}\AnimalTraits_dataset\AnimalTraits.csv'.format(main_path), sep=';')

animaltraits_bodymass = {}

for index, rows in df_animaltraits.iterrows():
        name = str(rows.species)
        if 'e_' in str(rows.body_mass): 
            main_num = str(rows.body_mass).split('e_0')
            body_mass = float(main_num[0]) / (10 ** float(main_num[1])) * 1000 
            df_animaltraits.at[index,'body_mass'] = body_mass
            
        else: 
            body_mass = round(float(rows.body_mass) * 1000, 2)
            df_animaltraits.at[index,'body_mass'] = body_mass
        animaltraits_bodymass.update({name: body_mass})
        
df_animaltraits = df_animaltraits.iloc[:, [5, 12]]

#### Results: List of organisms and their body mass (in g) from Animal Traits dataset

In [18]:
df_animaltraits

,species,body_mass
0,Hyla_versicolor,13.15
1,Pseudacris_triseriata,0.1
2,Acris_crepitans,0.3
3,Platymantis_vitiensis,0.3
4,Rana_sylvatica,0.37
...,...,...
3575,Sphaerodactylus_cinereus,0.54
3576,Sphaerodactylus_notatus,0.33
3577,Anolis_sagrei,0.4
3578,Coleonyx_switaki,9.46


### Script: Fill in missing body masses in AnAge with data from Animal Traits

In [19]:
def Sort(sub_li):
    return(sorted(sub_li, key = lambda x: x[1]))   

In [20]:
df_anage = pd.read_csv(r'{0}\AnAge_dataset\anage_data.csv'.format(main_path), sep=';')
organisms_downloaded = df_genes_organisms.columns.values.tolist()
  
for index, rows in df_anage.iterrows():
        name = str(rows.Genus) + '_' + str(rows.Species)
        if math.isnan(rows.Body_mass):
            if name in animaltraits_bodymass.keys():
                df_anage.at[index,'Body_mass'] = animaltraits_bodymass[name]
                
df_anage_animaltraits = df_anage.iloc[:, [6,7,20,28]]

### Results: AnAge and Animal Traits merged; table containing max. longevity and body mass

In [21]:
df_anage_animaltraits

,Genus,Species,Maximum_longevity,Body_mass
0,Daphnia,pulicaria,0.19,NaN
1,Drosophila,melanogaster,0.30,0.00095
2,Apis,mellifera,8.00,NaN
3,Cardiocondyla,obscurior,0.50,NaN
4,Lasius,niger,28.00,0.00058
...,...,...,...,...
4214,Scolymastra,joubini,15000.00,NaN
4215,Pinus,longaeva,5062.00,NaN
4216,Saccharomyces,cerevisiae,0.04,NaN
4217,Schizosaccharomyces,pombe,NaN,NaN


### Script: Calculate longevity quotient and choose low and top decile (aka short and long-living species)

In [22]:
organisms_in_organisms_lq_list =[]
organisms_lq_list_all = []
organisms_lq_list_final = []

# Go through the AnAge + Animal Traits table and calculate longevity quotient for each organism

for index, rows in df_anage_animaltraits.iterrows():
        name = str(rows.Genus) + '_' + str(rows.Species)
        if not math.isnan(rows.Body_mass) and not math.isnan(rows.Maximum_longevity):
           
            # maximum longevity
            ml = float(rows.Maximum_longevity)
            
            # body mass
            bm = float(rows.Body_mass)

            # longevity quotient
            lq = round(ml/(6.32*(bm**0.139)), 2)
            
            
            
            # All organisms from AnAge and Animal Traits datasets that data on body mass and lifespan exist
            
            organisms_lq_list_all.append({'organism':name, 'maximum_lifespan':ml, 'body_mass':bm, 'longevity_quotient':lq})
            
            
            
            # Overlap between list of organisms, that we have data for, and list of organisms with known bodymasses and lifespans
            
            if name in organisms_downloaded: 
                organisms_lq_list_final.append({'organism':name, 'maximum_lifespan':ml, 'body_mass':bm, 'longevity_quotient':lq})
        
        
# FINAL table of all organisms that data for orthologs and longevity quotient are available  
            
df_organisms_lq = pd.DataFrame.from_dict(organisms_lq_list_final)



# Calculate low and top decile of longevity quotients and save them as short and long-living organisms

df_organisms_lq['decile_rank'] = pd.qcut(df_organisms_lq['longevity_quotient'], 10, labels = False)

df_short_living = df_organisms_lq.loc[df_organisms_lq['decile_rank'].isin([0])]
short_living_list = df_short_living['organism'].to_list()

df_long_living = df_organisms_lq.loc[df_organisms_lq['decile_rank'].isin([9])]
long_living_list = df_long_living['organism'].to_list()


### Results: List of organisms sorted by decile rank
Low decile - short-living organisms

Top decile - long-living organisms

In [23]:
df_organisms_lq.sort_values('decile_rank')

,organism,maximum_lifespan,body_mass,longevity_quotient,decile_rank
85,Condylura_cristata,2.5,49.0,0.23,0
83,Sorex_araneus,3.2,8.4,0.38,0
77,Rattus_rattus,4.2,117.0,0.34,0
76,Rattus_norvegicus,3.8,206.9,0.29,0
75,Mus_musculus,4.0,18.0,0.42,0
...,...,...,...,...,...
61,Heterocephalus_glaber,31.0,35.3,2.99,9
56,Pan_troglodytes,59.4,43990.0,2.13,9
55,Homo_sapiens,90.0,70000.0,3.02,9
31,Myotis_lucifugus,34.0,5.8,4.21,9


### Results: List of short-living organisms

In [24]:
df_short_living

,organism,maximum_lifespan,body_mass,longevity_quotient,decile_rank
34,Antechinus_flavipes,3.9,46.5,0.36,0
36,Monodelphis_domestica,5.1,104.0,0.42,0
65,Arvicola_amphibius,2.5,94.7,0.21,0
66,Mesocricetus_auratus,3.9,108.2,0.32,0
75,Mus_musculus,4.0,18.0,0.42,0
76,Rattus_norvegicus,3.8,206.9,0.29,0
77,Rattus_rattus,4.2,117.0,0.34,0
83,Sorex_araneus,3.2,8.4,0.38,0
85,Condylura_cristata,2.5,49.0,0.23,0


### Results: List of long-living organisms

In [25]:
df_long_living

,organism,maximum_lifespan,body_mass,longevity_quotient,decile_rank
24,Desmodus_rotundus,29.2,29.4,2.89,9
27,Pteropus_giganteus,40.0,562.2,2.62,9
30,Eptesicus_fuscus,19.0,13.3,2.10,9
31,Myotis_lucifugus,34.0,5.8,4.21,9
32,Myotis_myotis,37.1,25.0,3.75,9
44,Tachyglossus_aculeatus,49.5,2909.0,2.58,9
55,Homo_sapiens,90.0,70000.0,3.02,9
56,Pan_troglodytes,59.4,43990.0,2.13,9
61,Heterocephalus_glaber,31.0,35.3,2.99,9


## 4 Select reference sequences 
**Description of script:** It chooses which protein sequence will be used as a reference sequence.

Sequence is chosen as a reference if there are keywords 'RefSeq Select' in GenPept file of the protein - script asks NCBI API for GenPept file and check it. Else the longest sequence from all protein sequences of that gene is chosen.

seq_selection: 
  1. 'only_one_seq'; there was only one reference sequence in NCBI dtb
  2. 'RefSeq_Select'; RefSeq Select in keywords of GenPept file
  3. 'longest_seq'; the longest protein sequence


### Variables and functions

In [26]:
selected_orthologs_list = [] 
# = list made of the 'orthologs_list' but containing only one seq per gene id
# + information if it was chosen with RefSeq Select keyword or it is the longest sequence
# [{'human_gene_name': 'X', 'gene_id': 'X', 'gene_name': 'X', 'organism'; 'X', 'accession_number': 'X', 'protein_sequence': 'X', 'seq_selection': 'X'}, {'human_gene_name': 'Y', 'gene_id': 'Y', 'gene_name': 'Y', 'organism'; 'Y', 'accession_number': 'Y', 'protein_sequence': 'Y', 'seq_selection': 'Y'}]


# testing dictionary 
# all_orthologs_refseqs_sorted = {'gene_id1': {'NP_001104745': {'protein_sequence':'X', 'human_gene_name': 'X', 'gene_name': 'X', 'organism': 'X'}, 'XP_023505037.1': {'protein_sequence':'X', 'human_gene_name': 'X', 'gene_name': 'X', 'organism': 'X'}}, 'gene_id2': {'XP_023505041.1' :{'protein_sequence':'X', 'human_gene_name': 'X', 'gene_name': 'X', 'organism': 'X'}}}


In [27]:
def chunks(data, SIZE=10000):
    it = iter(data)
    for i in range(0, len(data), SIZE):
        yield {k:data[k] for k in islice(it, SIZE)}

### Script: Parse dictionary all_orthologs_refseqs_sorted

In [28]:
#all refseqs from dictionary all_orthologs_refseqs_sorted

all_refseqs_list = [] 
gene_ids_list = []

for key in list(all_orthologs_refseqs_sorted.keys()):
    gene_ids_list.append(key)
    for jey in list(all_orthologs_refseqs_sorted[key].keys()): 
        all_refseqs_list.append(jey)
        
print('Count of all gene ids:', len(gene_ids_list))
print('Count of all refseqs:', len(all_refseqs_list), '\n')



# split the dictionary by 50 
splitted_all_orthologs_refseqs_sorted = []


for item in chunks({i:all_orthologs_refseqs_sorted[i] for i in all_orthologs_refseqs_sorted}, 10):
    splitted_all_orthologs_refseqs_sorted.append(item)
    
    
count_list = []
count = 0
splitted_refseqs_list = []
for item in splitted_all_orthologs_refseqs_sorted:
    for key in list(item.keys()):
        
        for jey in list(item[key].keys()):
            count+=1
            splitted_refseqs_list.append(jey)
    count_list.append(count)
    count = 0

print('Count in splitted:', len(splitted_refseqs_list))


print('Number of sub-parts of all_orthologs_refseqs_sorted to be examined:', len(splitted_all_orthologs_refseqs_sorted))
# print('Counts of refseqs in subparts:', count_list)



Count of all gene ids: 133845
Count of all refseqs: 351025 

Count in splitted: 351025
Number of sub-parts of all_orthologs_refseqs_sorted to be examined: 13385


### Script: Selection - one protein seq per gene id

In [ ]:
for chunk in splitted_all_orthologs_refseqs_sorted:
    
    
    refseqs_string = '' 
    refseqs_list = []
    
    
    for key in list(chunk.keys()):
        for jey in list(chunk[key].keys()): 
            refseqs_string = refseqs_string + '&id=' + jey
            refseqs_list.append(jey)
                    
    
    urls = requests.get("https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=protein{}&api_key=aabfba1a0deb347c0ffdd37c6e46908c6a09&rettype=gp".format(refseqs_string), auth=('user', 'pass'), timeout=(3.05, 27))
    genpept_files = urls.text.split('//')
    if '\n\n' in genpept_files: genpept_files.remove('\n\n')

    # print('Number of downloaded files is the same as number of RefSeq numbers:', len(genpept_files) == len(refseqs_list))
    # print(len(genpept_files), len(refseqs_list))
    
    count = -1
    for geneid, refseqs_of_geneid in chunk.items(): 
        refseq_select = None
        the_longest_seq = None
        seq_len = 0

        refseqs = list(refseqs_of_geneid.keys())
      
        # if there is more than one RefSeq sequence for gene id
        if len(refseqs) > 1 and count < len(genpept_files):
            for refseq in refseqs:
                count += 1
                genpept_file = genpept_files[count]
                if refseq not in genpept_file: print('Error: File does not correspond to RefSeq number.')

                # if the reference sequence is RefSeq Select, save it
                if 'RefSeq Select' in genpept_file:    
                    refseq_select = {'human_gene_name': refseqs_of_geneid[refseq]['human_gene_name'], 'gene_id': geneid, 'gene_name': refseqs_of_geneid[refseq]['gene_name'], 'organism': refseqs_of_geneid[refseq]['organism'], 'accession_number': refseq, 'protein_sequence': refseqs_of_geneid[refseq]['protein_sequence'], 'seq_selection':'RefSeq_Select'}


                # save the longest reference sequence 
                if len(refseqs_of_geneid[refseq]['protein_sequence']) > seq_len: 
                    seq_len = len(refseqs_of_geneid[refseq]['protein_sequence'])
                    the_longest_seq = {'human_gene_name': refseqs_of_geneid[refseq]['human_gene_name'], 'gene_id': geneid, 'gene_name': refseqs_of_geneid[refseq]['gene_name'], 'organism': refseqs_of_geneid[refseq]['organism'], 'accession_number': refseq, 'protein_sequence': refseqs_of_geneid[refseq]['protein_sequence'], 'seq_selection':'longest_seq'}


            if refseq_select: 
                selected_orthologs_list.append(refseq_select)

            elif the_longest_seq: 
                selected_orthologs_list.append(the_longest_seq)  
            

        else: # there is only one RefSeq
            count += 1
            refseq = list(refseqs_of_geneid.keys())[0]
            only_one_seq = {'human_gene_name': refseqs_of_geneid[refseq]['human_gene_name'], 'gene_id': geneid, 'gene_name': refseqs_of_geneid[refseq]['gene_name'], 'organism': refseqs_of_geneid[refseq]['organism'], 'accession_number': refseq, 'protein_sequence': refseqs_of_geneid[refseq]['protein_sequence'], 'seq_selection':'only_one_seq'}
            selected_orthologs_list.append(only_one_seq) 



df_selected_orthologs_list = pd.DataFrame.from_dict(selected_orthologs_list)

### Results: Table of selected reference sequences; one per gene id

In [ ]:
df_selected_orthologs_list

## 5 Create files for CAASTools Discovery analysis

### Script: Create config file with organisms and 0 (short-living) / 1 (long-living)

In [ ]:
config_file = open(r'{0}\CAAStools_files\config.txt'.format(main_path),"w")

for organism in short_living_list: 
    config_file.write(organism.lower() + '\t' + '0' + '\n')
for organism in long_living_list: 
    config_file.write(organism.lower() + '\t' + '1' + '\n')

config_file.close()

### Script: Create fasta file for clustal omega multiple sequence alignment

In [ ]:
all_seq = long_selected_orthologs_list + selected_orthologs_list
genes = df_genes_organisms.index.values.tolist()

for gene in genes:
    gene_file = open(r'{0}\CAAStools_files\fasta_files\{1}.fasta'.format(main_path, gene),"w")

    for item in all_seq:
        if gene in item.values(): 
            gene_file.write('>' + str(item['organism']).replace(' ', '_').lower() + '\n')
            gene_file.write(str(item['protein_sequence']) + '\n')

    gene_file.close

### Script: MSAs by Clustal omega
!!! It is needed to download the Clustal Omega program and save the folder "clustal-omega-1.2.2-win64" with the program "clustalo.exe" to main folder 

In [ ]:
for gene in genes:
    os.system('{0}\\clustal-omega-1.2.2-win64\\clustalo.exe -i {0}\\CAAStools_files\\fasta_files\\{1}.fasta --out {0}\\CAAStools_files\\MSA_results\\{1}.msa --outfmt clu --wrap=10000'.format(win_path, gene))

## 6 Create csv files

### stress_molecules.csv
table of stress molecules (downloaded from Reactome - with Reactome IDs requested, and checked with HGNC)

In [30]:
df_stress_molecules.to_csv(r'{0}\results\stress_molecules.csv'.format(main_path), sep=',', encoding='utf-8', index = False, header = False)

### multiple_paths_check.csv
table of gene symbols and Reactome IDs; i.e. some genes may be involved in multiple paths

In [31]:
multiple_paths_df.to_csv(r'{0}\results\gene_in_multiple_paths_check.csv'.format(main_path).format(main_path), sep=',', encoding='utf-8')

### orthologs.csv 
table of orthologs and their atributes (i.e. FASTA file parsed into table)

In [32]:
df_orthologs_list.to_csv(r'{0}\NCBI_Datasets_download\orthologs.csv'.format(main_path), sep=',', encoding='utf-8')      

### orthologs_count.csv
table of gene ids and their protein sequence counts

In [33]:
df_ortholog_count_list.to_csv(r'{0}\results\orthologs_count.csv'.format(main_path), sep=',', encoding='utf-8', index = False)

### genes_organisms_check.csv
table of genes-organisms check

In [34]:
df_genes_organisms.to_csv(r'{0}\results\genes_organisms_check.csv'.format(main_path), sep=',', encoding='utf-8')

### anage_animaltraits_data_merge.csv

In [35]:
df_anage_animaltraits.to_csv(r'{0}\results\anage_animaltraits_data_merge.csv'.format(main_path), sep=',', encoding='utf-8')

### organisms_lq.csv

In [36]:
df_organisms_lq.to_csv(r'{0}\results\organisms_lq.csv'.format(main_path), sep=',', encoding='utf-8')

### short_living_organisms.csv

In [37]:
df_short_living.to_csv(r'{0}\results\short_living_organisms.csv'.format(main_path), sep=',', encoding='utf-8', index = False)

### long_living_organisms.csv

In [38]:
df_long_living.to_csv(r'{0}\results\long_living_organisms.csv'.format(main_path), sep=',', encoding='utf-8', index = False)

### final_dataset.csv
table of selected sequences one reference sequence per gene id (i.e. orthologous gene)

In [ ]:
df_selected_orthologs_list.to_csv(r'{0}\results\final_dataset.csv'.format(main_path), sep=',', encoding='utf-8')

## 7 Additional: Reference sequences selection if NCBI connection error
Sometimes, there is an error when querying NCBI that was not resolved. With this part of script it is possible to get specific parts (i.e. get reference sequences one per gene) of dataset manually.

To slow down the querying process, the downloaded ortholog FASTA files (in folder NCBI Datasets download) are splited into multiple folders and the FASTA files are read again.

### Divide NCBI downloaded files into folders

In [ ]:
folder = os.listdir(r'{0}\NCBI_Datasets_download'.format(main_path))
if '.ipynb_checkpoints' in folder: folder.remove('.ipynb_checkpoints')
name = 1
counter = 0


os.system('mkdir {0}\\NCBI_Datasets_download\\{1}'.format(win_path, str(name)))
          
for gene in folder:
    if counter < 20:
        os.system('move {0}\\NCBI_Datasets_download\\{1} {0}\\NCBI_Datasets_download\\{2}'.format(win_path, gene, str(name)))
        counter += 1
        
    else:
        name += 1
        os.system('mkdir {0}\\NCBI_Datasets_download\\{1}'.format(win_path, str(name)))
        counter = 0
        os.system('move {0}\\NCBI_Datasets_download\\{1} {0}\\NCBI_Datasets_download\\{2}'.format(win_path, gene, str(name)))
        counter += 1

### Parse FASTA files containing orthologs transcript sequences and select one reference sequence per organism 

#### Variables

In [40]:
selected_orthologs_list = [] 

undone_list = []

def most_frequent_count(List):
        occurence_count = Counter(List)
        return occurence_count.most_common(1)[0][1]

def most_frequent_name(List):
        occurence_count = Counter(List)
        return occurence_count.most_common(1)[0][0]

def chunks(data, SIZE=10000):
        it = iter(data)
        for i in range(0, len(data), SIZE):
            yield {k:data[k] for k in islice(it, SIZE)}

#### Script: go through all folders containing orthologs FASTA files
1. Parse FASTA files and save the info
2. Choose one reference sequence per organism

In [60]:
folders = os.listdir(r'{0}\NCBI_Datasets_download'.format(main_path))
if '.ipynb_checkpoints' in folders: folders.remove('.ipynb_checkpoints')


for folder in folders:
    print(folder)


    #### 1. Parse FASTA files and save the information into lists and dictionaries
            # parse all FASTA files downloaded 
            # every file is named as the gene which orthologs seqs it contains
    
    files = os.listdir(r'{0}\NCBI_Datasets_download\{1}'.format(main_path, folder))
    if '.ipynb_checkpoints' in files:
        files.remove('.ipynb_checkpoints')


    print(files)
    
    all_orthologs_refseqs_sorted = {}

    # {gene_id1: 
    #     {RefSeq1: 
    #          {protein_sequence: X, human_gene_name: X, gene_name: X, organism: X}, 
    #      RefSeq2: {...}
    #     }, 
    #  gene_id2: 
    #      {RefSeq3: {}
    #     }, 
    #  ...}

    

    for file in files:
        already_been_there = []


        gene_name = file.replace('.faa', '')

        with open(r'{0}\NCBI_Datasets_download\{1}\{2}'.format(main_path, folder, file)) as fasta_file:
            parser = fastaparser.Reader(fasta_file)

            for seq in parser: # for every record (ortholog) of FASTA file        

                # split the FASTA record 
                working_list = seq.description.split(" [")     
                gene_id = (re.findall('\\d+', working_list[2])[0])
                # gene_ids.append(gene_id)

                organism_name = working_list[1][9:].replace("]", "")
                organism_name = organism_name.replace(" ", "_")
                


            
                if gene_id not in already_been_there: 
                    first_sequence = seq.sequence_as_string()
                    all_orthologs_refseqs_sorted[gene_id] = {}
                    all_orthologs_refseqs_sorted[gene_id].update({seq.id: {'protein_sequence': seq.sequence_as_string(), 'human_gene_name': gene_name, 'gene_name': working_list[0], 'organism': organism_name}})

                else:
                    another_sequence = seq.sequence_as_string()
                    all_orthologs_refseqs_sorted[gene_id].update({seq.id: {'protein_sequence': seq.sequence_as_string(), 'human_gene_name': gene_name, 'gene_name': working_list[0], 'organism': organism_name}})

                already_been_there.append(gene_id)

    

    
    #### 2. Parse dictionary all_orthologs_refseqs_sorted for easier querying the server
    
    # all refseqs from dictionary all_orthologs_refseqs_sorted
    all_refseqs_list = [] 
    gene_ids_list = []

    for key in list(all_orthologs_refseqs_sorted.keys()):
        gene_ids_list.append(key)
        for jey in list(all_orthologs_refseqs_sorted[key].keys()): 
            all_refseqs_list.append(jey)

    print('Count of all gene ids:', len(gene_ids_list))
    print('Count of all refseqs:', len(all_refseqs_list), '\n')



    # split the dictionary 
    splitted_all_orthologs_refseqs_sorted = []


    for item in chunks({i:all_orthologs_refseqs_sorted[i] for i in all_orthologs_refseqs_sorted}, 5):
        splitted_all_orthologs_refseqs_sorted.append(item)


    count_list = []
    count = 0
    splitted_refseqs_list = []
    for item in splitted_all_orthologs_refseqs_sorted:
        for key in list(item.keys()):

            for jey in list(item[key].keys()):
                count+=1
                splitted_refseqs_list.append(jey)
        count_list.append(count)
        count = 0

    print('Count in splitted:', len(splitted_refseqs_list))
    print('Number of sub-parts of all_orthologs_refseqs_sorted to be examined:', len(splitted_all_orthologs_refseqs_sorted))

    
    #### 3. Selection - one protein seq per gene id

    for chunk in splitted_all_orthologs_refseqs_sorted:


        refseqs_string = '' 
        refseqs_list = []


        for key in list(chunk.keys()):
            for jey in list(chunk[key].keys()): 
                refseqs_string = refseqs_string + '&id=' + jey
                refseqs_list.append(jey)

                
        session = requests.Session()
        retry = Retry(connect=5, backoff_factor=1.5)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount('http://', adapter)
        session.mount('https://', adapter)

        genpept_files = session.get("https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=protein{}&api_key=aabfba1a0deb347c0ffdd37c6e46908c6a09&rettype=gp".format(refseqs_string))
        
        
        genpept_files = genpept_files.text.split('//')
        if '\n\n' in genpept_files: genpept_files.remove('\n\n')


        count = -1
        for geneid, refseqs_of_geneid in chunk.items(): 
            refseq_select = None
            the_longest_seq = None
            seq_len = 0

            refseqs = list(refseqs_of_geneid.keys())

            # if there is more than one RefSeq sequence for gene id
            if len(refseqs) > 1 and count < len(genpept_files):
                for refseq in refseqs:
                    count += 1
                    genpept_file = genpept_files[count]
                    if refseq not in genpept_file: print('Error: File does not correspond to RefSeq number.')

                    # if the reference sequence is RefSeq Select, save it
                    if 'RefSeq Select' or 'MANE Select' in genpept_file:    
                        refseq_select = {'human_gene_name': refseqs_of_geneid[refseq]['human_gene_name'], 'gene_id': geneid, 'gene_name': refseqs_of_geneid[refseq]['gene_name'], 'organism': refseqs_of_geneid[refseq]['organism'], 'accession_number': refseq, 'protein_sequence': refseqs_of_geneid[refseq]['protein_sequence'], 'seq_selection':'RefSeq_Select'}


                    # save the longest reference sequence 
                    if len(refseqs_of_geneid[refseq]['protein_sequence']) > seq_len: 
                        seq_len = len(refseqs_of_geneid[refseq]['protein_sequence'])
                        the_longest_seq = {'human_gene_name': refseqs_of_geneid[refseq]['human_gene_name'], 'gene_id': geneid, 'gene_name': refseqs_of_geneid[refseq]['gene_name'], 'organism': refseqs_of_geneid[refseq]['organism'], 'accession_number': refseq, 'protein_sequence': refseqs_of_geneid[refseq]['protein_sequence'], 'seq_selection':'longest_seq'}


                if refseq_select: 
                    selected_orthologs_list.append(refseq_select)

                elif the_longest_seq: 
                    selected_orthologs_list.append(the_longest_seq)  


            else: # there is only one RefSeq
                count += 1
                refseq = list(refseqs_of_geneid.keys())[0]
                only_one_seq = {'human_gene_name': refseqs_of_geneid[refseq]['human_gene_name'], 'gene_id': geneid, 'gene_name': refseqs_of_geneid[refseq]['gene_name'], 'organism': refseqs_of_geneid[refseq]['organism'], 'accession_number': refseq, 'protein_sequence': refseqs_of_geneid[refseq]['protein_sequence'], 'seq_selection':'only_one_seq'}
                selected_orthologs_list.append(only_one_seq) 

    df_selected_orthologs_list = pd.DataFrame.from_dict(selected_orthologs_list)
    os.system('move {0}\\NCBI_Datasets_download\\{1} {0}\\NCBI_Datasets_download_done'.format(win_path, folder))


17
['LAMTOR1.faa', 'LAMTOR2.faa', 'LAMTOR3.faa', 'LAMTOR4.faa', 'LAMTOR5.faa', 'LIG1.faa', 'LIG3.faa', 'LIG4.faa', 'LMNA.faa', 'MAD2L2.faa', 'MAP1LC3A.faa', 'MAP1LC3B.faa', 'MAP1LC3C.faa', 'MAPK1.faa', 'MAPK3.faa', 'MAPK8.faa', 'MAPKAPK2.faa', 'MBD4.faa', 'MBTPS1.faa', 'MBTPS2.faa']
Count of all gene ids: 3902
Count of all refseqs: 9312 

Count in splitted: 9312
Number of sub-parts of all_orthologs_refseqs_sorted to be examined: 781
18
['MCRS1.faa', 'MDC1.faa', 'MFN1.faa', 'MFN2.faa', 'MGMT.faa', 'MIOS.faa', 'MLH1.faa', 'MLST8.faa', 'MNAT1.faa', 'MPG.faa', 'MRE11.faa', 'MSH2.faa', 'MSH3.faa', 'MSH6.faa', 'MTERF3.faa', 'MTMR14.faa', 'MTMR3.faa', 'MTOR.faa', 'MUS81.faa', 'MUTYH.faa']
Count of all gene ids: 3997
Count of all refseqs: 14581 

Count in splitted: 14581
Number of sub-parts of all_orthologs_refseqs_sorted to be examined: 800
19
['MVB12A.faa', 'MVB12B.faa', 'MYDGF.faa', 'NBN.faa', 'NBR1.faa', 'NDC1.faa', 'NEIL1.faa', 'NEIL2.faa', 'NEIL3.faa', 'NFRKB.faa', 'NFYA.faa', 'NFYB.faa'

IndexError: list index out of range

### If NCBI error: MANUALLY save the processed parts

#### All checked genes from the last run of program:

In [61]:
df_selected_orthologs_list = pd.DataFrame.from_dict(selected_orthologs_list)

In [62]:
df_selected_orthologs_list

,human_gene_name,gene_id,gene_name,organism,accession_number,protein_sequence,seq_selection
0,AAAS,100063811,AAAS,Equus_caballus,XP_023499419.1,MRVAPSFSPVHSTGRDVGLFGVLNEIANSEEEVFEWVKTASSWALA...,RefSeq_Select
1,AAAS,100154333,AAAS,Sus_scrofa,XP_003355451.1,MCSLGLFPPPPPRGQVTLYEHNNELVTGSSYESPPPDFRGQWINLP...,only_one_seq
2,AAAS,100356621,AAAS,Oryctolagus_cuniculus,XP_051701511.1,MARGLPSSITGSKCGKDASTRVFEWVKTASSWALALCRWASSLHGS...,RefSeq_Select
3,AAAS,100405154,AAAS,Callithrix_jacchus,XP_054094472.1,MKLQTQKKRSCFLNPTVFEWVKTASGWALALCRWASSLHGSLFPHL...,RefSeq_Select
4,AAAS,100444879,AAAS,Pongo_abelii,XP_054383964.1,MALCQWASSLHGSLFPHLSLRSEDLIAEFAQVTNWSSCCLRVFAWH...,RefSeq_Select
...,...,...,...,...,...,...,...
109053,USP7,123844448,USP7,Mirounga_angustirostris,XP_045730909.1,MVMPRFYPDRPHQKSVGFFLQCNAESDSTSWSCHAQAVLKIINYRD...,RefSeq_Select
109054,USP7,123933888,USP7,Meles_meles,XP_045849557.1,MAWSEVTDPEKGFIDDDKVTFEVFVQADAPHGVAWDSKKHTGYVGL...,RefSeq_Select
109055,USP7,124093097,Usp7,Marmota_monax,XP_046301246.1,MVMPRFYPDRPHQKSVGFFLQCNAESDSTSWSCHAQAVLKIINYRD...,RefSeq_Select
109056,USP7,124241803,USP7,Equus_quagga,XP_046521936.1,MVMPRFYPDRPHQKSVGFFLQCNAESDSTSWSCHAQAVLKIINYRD...,RefSeq_Select


#### Merge all runs 

In [66]:
selected_orthologs_list += undone_list

### All already checked sequences (all runs)

In [67]:
df_selected_orthologs_list = pd.DataFrame.from_dict(selected_orthologs_list)

In [68]:
df_selected_orthologs_list

,human_gene_name,gene_id,gene_name,organism,accession_number,protein_sequence,seq_selection
0,AAAS,100063811,AAAS,Equus_caballus,XP_023499419.1,MRVAPSFSPVHSTGRDVGLFGVLNEIANSEEEVFEWVKTASSWALA...,RefSeq_Select
1,AAAS,100154333,AAAS,Sus_scrofa,XP_003355451.1,MCSLGLFPPPPPRGQVTLYEHNNELVTGSSYESPPPDFRGQWINLP...,only_one_seq
2,AAAS,100356621,AAAS,Oryctolagus_cuniculus,XP_051701511.1,MARGLPSSITGSKCGKDASTRVFEWVKTASSWALALCRWASSLHGS...,RefSeq_Select
3,AAAS,100405154,AAAS,Callithrix_jacchus,XP_054094472.1,MKLQTQKKRSCFLNPTVFEWVKTASGWALALCRWASSLHGSLFPHL...,RefSeq_Select
4,AAAS,100444879,AAAS,Pongo_abelii,XP_054383964.1,MALCQWASSLHGSLFPHLSLRSEDLIAEFAQVTNWSSCCLRVFAWH...,RefSeq_Select
...,...,...,...,...,...,...,...
1658793,REV3L,101663394,REV3L,Echinops_telfairi,XP_004702065.1,MAFSIDRALNVALGNPSSTAQHVFKVSLVSGMPFYGYHEKERHFMK...,only_one_seq
1658794,REV3L,101673076,REV3L,Mustela_putorius_furo,XP_044936137.1,MAFSIDRALNVALGNPSSTAQHVFKVSLVSGMPFYGYHEKERHFMK...,RefSeq_Select
1658795,REV3L,101723274,Rev3l,Heterocephalus_glaber,XP_021107698.1,MAFSIDRALNVALGNPSSTAQHVFKVSLVSGMPFYGYHEKERHFMK...,RefSeq_Select
1658796,REV3L,101787733,Rev3l,Cavia_porcellus,XP_023419436.1,MFSVRIVTADYYMASPLQGLDICQSPLTQIPVKKVPVVRVFGATPA...,only_one_seq


#### Save the table of existing results; all seqs that have been checked already

In [ ]:
number_of_try = str(input('Type in the number of try:'))

In [ ]:
df_selected_orthologs_list.to_csv(r'{0}\results\ncbi_err_selected_orthologs{1}.csv'.format(main_path, number_of_try), sep=',', encoding='utf-8')

#### Save the sub-dataset

In [ ]:
undone_list = selected_orthologs_list